# Summarize MODFLOW output
The following script will summarize steady state groundwater flux outputted from a MODFLOW run.

## Load Python packages and import data

In [1]:
import os
import numpy as np
import shapefile
import pickle
from shapely.geometry import Polygon
from simpledbf import Dbf5
import flopy.utils.binaryfile as bf

PyTables is not installed. No support for HDF output.


### Set file name to MODFLOW flux file (_*.flx_) the user wishes to summarize

In [2]:
modflowdir = '../MODFLOW/'
mdlprfx = 'SHModel-CH_310-NWT-Raven2025'
modflowflxfp = "../MODFLOW/"+mdlprfx+".flx"


### Set file name of polygons the user wished to summarize groundwater discharge to streams

In [3]:
polygonfp = "../GIS/CH_Subs_Raven2025.shp"

### Set output raster definition

In [4]:
# (keep consistent with Raven raster definition)
nrows = 719
ncols = 726
xul = 1323330
yul = 11906070
cs = 60. # grid cell width

### Import MODFLOW model cell to CH sub-basin cross-reference
> do not change

In [5]:
df = Dbf5("../GIS/SHModel-CH_310_centroids.dbf").to_dataframe().dropna().astype('int32')
dsws = dict(zip(df['CellID'], df['SubId']))
for i in range(885*890):
    if i in dsws: continue
    dsws[i]=-9999
dsws = dict(sorted(dsws.items()))
sws = np.array(list(dsws.values()),dtype=np.int16).reshape((885,890))

### Import Sub-basin polygons and collect areas

In [6]:
sf = shapefile.Reader(polygonfp)
geom = sf.shapes()
attr = sf.records()
darea = dict()
for i in range(len(geom)):
    plgn = Polygon(geom[i].points)
    darea[int(attr[i]['SubId'])] = plgn.area

## Read MODFLOW output steady-state fluxes

Groundwater discharge to streams is represented in the model using 2 head-dependent flux boundary packages: 
1. `DRAINS` -- from the [DRN](https://water.usgs.gov/ogw/modflow-nwt/MODFLOW-NWT-Guide/drn.html) package; and
1. `RIVER LEAKAGE` -- from the [RIV](https://water.usgs.gov/ogw/modflow-nwt/MODFLOW-NWT-Guide/riv.html) package.

In [7]:
flx = bf.CellBudgetFile(modflowflxfp)
# print(flx.headers) # print available data

rdrn = flx.get_data(kstpkper=(0,0), text='DRAINS', full3D=True)
adrn = np.sum(np.array(rdrn[0]), axis=0)
rriv = flx.get_data(kstpkper=(0,0), text='RIVER LEAKAGE', full3D=True)
ariv = np.sum(np.array(rriv[0]), axis=0)

gwd = -(adrn+ariv)*365.24*86400*1000 # convert total drainage from m3/s to mm·m2/yr

dgwd = dict()
for sid in set(dsws.values()):
    if sid==-9999: continue
    dgwd[sid] = np.sum(gwd[sws==sid])
    print("sub-basin {}: {:.1f} mm/yr".format(sid, dgwd[sid]/darea[sid]))

sub-basin 1: 45.3 mm/yr
sub-basin 2: 64.6 mm/yr
sub-basin 3: 167.2 mm/yr
sub-basin 4: 21.1 mm/yr
sub-basin 5: -5.5 mm/yr
sub-basin 6: 134.6 mm/yr
sub-basin 7: 58.4 mm/yr
sub-basin 8: 149.0 mm/yr
sub-basin 9: 95.8 mm/yr
sub-basin 10: 77.1 mm/yr
sub-basin 11: 4.2 mm/yr
sub-basin 12: 252.3 mm/yr
sub-basin 13: 0.0 mm/yr
sub-basin 14: 61.9 mm/yr
sub-basin 15: 38.8 mm/yr
sub-basin 16: 114.0 mm/yr
sub-basin 17: 232.9 mm/yr
sub-basin 18: 15.5 mm/yr
sub-basin 19: 220.8 mm/yr
sub-basin 20: 41.2 mm/yr
sub-basin 21: 116.8 mm/yr
sub-basin 22: 121.3 mm/yr
sub-basin 23: 58.0 mm/yr
sub-basin 24: 58.8 mm/yr
sub-basin 25: 244.3 mm/yr
sub-basin 26: 15.5 mm/yr
sub-basin 27: 110.6 mm/yr
sub-basin 28: 47.7 mm/yr
sub-basin 29: 124.9 mm/yr
sub-basin 30: 152.7 mm/yr
sub-basin 31: -27.2 mm/yr
sub-basin 32: 85.3 mm/yr
sub-basin 33: 118.0 mm/yr
sub-basin 34: 12.7 mm/yr
sub-basin 35: 87.7 mm/yr
sub-basin 36: 8.8 mm/yr
sub-basin 37: -11.3 mm/yr
sub-basin 39: 81.1 mm/yr
sub-basin 40: 163.2 mm/yr
sub-basin 41: 31.8 m

### Summarize over gauges catchments

In [8]:
dgag = {'02HB004': [23,24,25,28,33,57,68,69],
        '02HB005': [40,56,65,66,67],
        '02HB011': [6,16,17,19,22,48,59,60,61,62,63,64],
        '02HB012': [21,29,30,31,35,47,50,53,54],
        '02HB022': [48,61,62,63,64],
        '02HB027': [58],
        '02HB028': [30,47,50],
        '02HB032': [62,64],
        '02HB033': [62,63,64]}

for gag,sids in dgag.items():
    sarea,sflx = 0., 0.
    for s in sids:
        sarea+=darea[s]
        sflx+=dgwd[s]
    print(gag, sflx/sarea)


02HB004 88.49169
02HB005 126.997604
02HB011 172.18974
02HB012 151.32932
02HB022 147.98317
02HB027 63.92938
02HB028 202.83524
02HB032 104.64441
02HB033 119.85792


## Create GIS raster from MODFLOW output

MODFLOW to raster mapping

In [9]:
mapfp = '../scripts/supp/SHModel-CH_310_to_Raven2025_map.pkl' # used to map MODFLOW cells to Raven HRUs

In [10]:
def dictToNp(dat, nr, nc, nodata=-9999.):
    def crc(c): # cell to row-col
        i = int(c/nc)
        j = c - i*nc
        return (i,j)    

    a = np.full((nr,nc),nodata,dtype=np.float32)
    for cid,v in dat.items(): a[crc(cid)] = np.float32(v)
    return a


def saveBinaryFloat(fp, dat):
    fn, _ = os.path.splitext(fp)
    with open(fn+'.hdr', 'w') as f: # save header
        f.write('BYTEORDER      I\n')
        f.write('LAYOUT         BIL\n')
        f.write('NROWS          '+str(nrows)+'\n')
        f.write('NCOLS          '+str(ncols)+'\n')
        f.write('NBANDS         1\n')
        f.write('NBITS          32\n')
        f.write('BANDROWBYTES   '+str(nrows*4)+'\n')
        f.write('TOTALROWBYTES  '+str(nrows*4)+'\n')
        f.write('PIXELTYPE      FLOAT\n')
        f.write('ULXMAP         '+str(xul+cs/2)+'\n') # The x-axis map coordinate of the center of the upper-left pixel.
        f.write('ULYMAP         '+str(yul-cs/2)+'\n') # The y-axis map coordinate of the center of the upper-left pixel.
        f.write('XDIM           '+str(cs)+'\n')
        f.write('YDIM           '+str(cs)+'\n')
        f.write('NODATA         -9999\n')

    dat = dat.copy()
    if type(dat)==dict: dat = dictToNp(dat,nrows,ncols)
    if os.path.exists(fp): os.remove(fp)
    dat.tofile(fp) # always saved in C-order (row-major)    

In [11]:
with open(mapfp,'rb') as f: gmap = pickle.load(f)
dgwd, wgwd = dict(), dict()
agwd = gwd.reshape(885*890)
for c,ws in gmap.items():
    for cMODFLOW,w in ws.items():
        if c in dgwd:
            dgwd[c]+=w*agwd[cMODFLOW]
            wgwd[c]+=w
        else:
            dgwd[c]=w*agwd[cMODFLOW]
            wgwd[c]=w

for c in gmap: dgwd[c]/=wgwd[c]*cs*cs
print(' mean discharge to streams: '+str(round(sum(dgwd.values()) / len(dgwd),1))+' mm/yr')
saveBinaryFloat(modflowdir+mdlprfx+'_groundwater-discharge.bil',dgwd)

 mean discharge to streams: 70.9 mm/yr
